In [14]:
# Retail Sales Data Analysis and Exploration
## Project Overview
"""
This notebook contains the initial exploration of retail sales data for forecasting purposes. The analysis focuses on understanding 
data patterns, quality, and potential features for our forecasting model.    
"""
# Step 1: Import all necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn  # Add this basic import
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import plotly.express as px
from datetime import datetime
import plotly.graph_objs as go
import plotly.subplots as sp
import warnings
import os

# Ignore warnings
warnings.filterwarnings('ignore')

# Display settings

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.expand_frame_repr', False)

# Verify your setup - now this will work!
print(f"Python version: {pd.__version__}")
print(f"NumPy version: {np.__version__}")
print(f"Scikit-learn version: {sklearn.__version__}")

Python version: 2.2.3
NumPy version: 2.0.2
Scikit-learn version: 1.5.2


In [15]:
current_dir = os.getcwd()
print(f"Current working directory: {current_dir}")
def load_data():
    """
    Load the retail sales dataset and perform initial datetime conversion.
    """
    df = pd.read_csv('/xxxxxxxxxxxxx')
    df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
    return df

def assess_data_quality(df):
    """
    Perform initial data quality checks.
    """
    print("Missing Value Analysis")
    print("-" * 50)
    missing_stats = pd.DataFrame({
        'Missing Values': df.isnull().sum(),
        'Percentage Missing': (df.isnull().sum() / len(df) * 100).round(2)
    })
    print(missing_stats[missing_stats['Missing Values'] > 0])
    
    print("\nDuplicate Records Analysis")
    print("-" * 50)
    duplicates = df.duplicated().sum()
    print(f"Number of duplicate records: {duplicates:,}")
    
    return missing_stats

def handle_missing_values(df):
    """
    Handle missing values in the retail dataset using specific strategies.
    """
    df_clean = df.copy()
    
    # Fill WarehouseLocation with mode
    df_clean['WarehouseLocation'] = df_clean['WarehouseLocation'].fillna(
        df_clean['WarehouseLocation'].mode()[0]
    )
    
    # Fill ShippingCost with country-wise median
    df_clean['ShippingCost'] = df_clean.groupby('Country')['ShippingCost'].transform(
        lambda x: x.fillna(x.median())
    )

    # Generate random guest customer IDs starting from 90000
    # This ensures they're distinguishable from regular customer IDs
    missing_customer_count = df_clean['CustomerID'].isnull().sum()
    guest_ids = np.arange(90000, 90000 + missing_customer_count)
    
    # Fill missing CustomerIDs with these guest IDs
    df_clean.loc[df_clean['CustomerID'].isnull(), 'CustomerID'] = guest_ids
    
    return df_clean

# Main execution sequence
if __name__ == "__main__":
    # 1. Load the data
    df = load_data()
    print("Data loaded successfully")
    
    # 2. Assess data quality
    missing_stats = assess_data_quality(df)
    
    # 3. Handle missing values
    df_clean = handle_missing_values(df)
    
    # 4. Verify cleaning results
    print("\nMissing Values After Cleaning")
    print("-" * 50)
    print(df_clean.isnull().sum())
    
    # 5. Save processed dataset
    output_dir = os.path.join(current_dir, 'data', 'processed')
    os.makedirs(output_dir, exist_ok=True)
    output_path = os.path.join(output_dir, 'retail_sales_2024_cleaned.csv')
    df_clean.to_csv(output_path, index=False)
    print(f"Full path of saved file: {output_path}")

Current working directory: /xxxxxxxxx
Data loaded successfully
Missing Value Analysis
--------------------------------------------------
                   Missing Values  Percentage Missing
CustomerID                   4978                10.0
ShippingCost                 2489                 5.0
WarehouseLocation            3485                 7.0

Duplicate Records Analysis
--------------------------------------------------
Number of duplicate records: 0

Missing Values After Cleaning
--------------------------------------------------
InvoiceNo            0
StockCode            0
Description          0
Quantity             0
InvoiceDate          0
UnitPrice            0
CustomerID           0
Country              0
Discount             0
PaymentMethod        0
ShippingCost         0
Category             0
SalesChannel         0
ReturnStatus         0
ShipmentProvider     0
WarehouseLocation    0
OrderPriority        0
dtype: int64
Full path of saved file: /xxxxxxxxx

In [16]:
def load_cleaned_data():
    """
    Load the cleaned retail dataset and perform datetime conversion.
    
    Returns:
        pd.DataFrame: The cleaned dataset with converted datetime
    """
    cleaned_path = 'xxxxxxxxxx'
    df = pd.read_csv(cleaned_path)
    df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
    return df

In [28]:
import pandas as pd
import os
from datetime import datetime

def load_cleaned_data(file_path):
    """
    Load and validate the cleaned retail dataset.
    
    Args:
        file_path (str): Path to the cleaned CSV file
        
    Returns:
        pd.DataFrame: Loaded and validated dataframe
    """
    try:
        df = pd.read_csv(file_path)
        df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
        print("Data loaded successfully")
        return df
    except Exception as e:
        print(f"Error loading data: {str(e)}")
        raise

def create_optimized_tables():
    """
    Create optimized fact and dimension tables based on cardinality analysis.
    """
    try:
        # Load cleaned data
        input_path = 'xxxxxxxxx'
        df = load_cleaned_data(input_path)
        
        # Create fact table focusing on numerical metrics
        fact_sales = df[[
            'InvoiceNo',
            'StockCode',
            'CustomerID',
            'InvoiceDate',
            'Quantity',
            'UnitPrice',
            'Discount',
            'ShippingCost'
        ]].copy()
        
        # Calculate total amount with discount
        fact_sales['TotalAmount'] = (fact_sales['Quantity'] * 
                                   fact_sales['UnitPrice'] * 
                                   (1 - fact_sales['Discount']))
        
        # Format InvoiceDate consistently
        fact_sales['InvoiceDate'] = fact_sales['InvoiceDate'].dt.strftime('%Y-%m-%d')
        
        # Create dimension tables
        dim_product = df[[
            'StockCode',
            'Description',
            'Category'
        ]].drop_duplicates()
        
        dim_customer = df[[
            'CustomerID',
            'Country',
            'SalesChannel'
        ]].drop_duplicates()
        
        dim_shipment = df[[
            'InvoiceNo',
            'ShipmentProvider',
            'WarehouseLocation',
            'OrderPriority',
            'ReturnStatus',
            'PaymentMethod'
        ]].drop_duplicates()
        
        # Create date dimension with matching format
        dim_date = pd.DataFrame({
            'InvoiceDate': df['InvoiceDate'].dt.strftime('%Y-%m-%d').unique()
        })
        
        # Add date attributes
        dim_date['DateForCalculation'] = pd.to_datetime(dim_date['InvoiceDate'])
        dim_date['Year'] = dim_date['DateForCalculation'].dt.year
        dim_date['Month'] = dim_date['DateForCalculation'].dt.month
        dim_date['Quarter'] = dim_date['DateForCalculation'].dt.quarter
        dim_date['DayOfWeek'] = dim_date['DateForCalculation'].dt.dayofweek
        dim_date['IsWeekend'] = dim_date['DayOfWeek'].isin([5, 6]).astype(int)
        dim_date = dim_date.drop('DateForCalculation', axis=1)
        
        # Save all tables
        output_dir = 'xxxxxxx'
        
        fact_sales.to_csv(f'{output_dir}/fact_sales.csv', index=False)
        dim_product.to_csv(f'{output_dir}/dim_product.csv', index=False)
        dim_customer.to_csv(f'{output_dir}/dim_customer.csv', index=False)
        dim_shipment.to_csv(f'{output_dir}/dim_shipment.csv', index=False)
        dim_date.to_csv(f'{output_dir}/dim_date.csv', index=False)
        
        print("All tables created successfully")
        
    except Exception as e:
        print(f"Error creating tables: {str(e)}")
        raise

def main():
    """
    Execute the creation of optimized tables and provide documentation.
    """
    try:
        # Create output directory
        output_dir = 'xxxxxxxx'
        os.makedirs(output_dir, exist_ok=True)
        
        # Create all tables
        create_optimized_tables()
        
        # Print documentation
        print("\nOptimized tables created for Tableau analysis:")
        print("\nfact_sales.csv - Core transaction data:")
        print("- Contains all numerical metrics and high-cardinality identifiers")
        print("- Enables detailed transaction-level analysis")
        
        print("\ndim_product.csv - Product information:")
        print("- Links products to their descriptions and categories")
        print("- Supports product hierarchy analysis")
        
        print("\ndim_customer.csv - Customer details:")
        print("- Contains customer-related attributes")
        print("- Enables customer segmentation analysis")
        
        print("\ndim_shipment.csv - Shipment and order details:")
        print("- Consolidates all shipping-related attributes")
        print("- Supports order fulfillment analysis")
        
        print("\ndim_date.csv - Time dimension:")
        print("- Provides hierarchical time attributes")
        print("- Enables temporal pattern analysis")
        
    except Exception as e:
        print(f"Error in main execution: {str(e)}")
        raise

if __name__ == "__main__":
    main()

Data loaded successfully
All tables created successfully

Optimized tables created for Tableau analysis:

fact_sales.csv - Core transaction data:
- Contains all numerical metrics and high-cardinality identifiers
- Enables detailed transaction-level analysis

dim_product.csv - Product information:
- Links products to their descriptions and categories
- Supports product hierarchy analysis

dim_customer.csv - Customer details:
- Contains customer-related attributes
- Enables customer segmentation analysis

dim_shipment.csv - Shipment and order details:
- Consolidates all shipping-related attributes
- Supports order fulfillment analysis

dim_date.csv - Time dimension:
- Provides hierarchical time attributes
- Enables temporal pattern analysis
